In [1]:
from gqimax.mapper import map_cx, map_noncx
from gqimax.sample import sample2
from gqimax.mapper import weightss_to_lambdas, map_indices_to_indicess, map_noncx
import cupy as cp
import time

import cupy as cp

# def weightsss_to_lambdass(lambdass: list, weightsss: list) -> cp.ndarray:
#     n = len(weightsss)
#     streams = [cp.cuda.Stream() for _ in range(n)]
#     mapped_lambdass = [None] * n
#     non_zeros_indicess = [None] * n
#     for i in range(n):
#         with streams[i]:
#             new_lambdas, non_zeros_indices = weightss_to_lambdas(
#                 lambdass[i], weightsss[i]
#             )
#             mapped_lambdass[i] = new_lambdas
#             non_zeros_indicess[i] = non_zeros_indices
#     for stream in streams:
#         stream.synchronize()
    
#     return mapped_lambdass, map_indices_to_indicess(non_zeros_indicess)

num_qubits = 4
ins = sample2(num_qubits, 3)
ins.operatoring()
from gqimax.pstabilizers import PStabilizers
stb = PStabilizers(num_qubits)
lambdass = stb.lambdass
indicesss = stb.indicess

for j, order in enumerate(ins.orders):
    k = j // 2
    if order == 0:
        lambdass, indicesss = map_noncx(lambdass, indicesss, ins.lut[k])

    else:
        for _, indices, _ in ins.xoperators[k]:
            lambdass, indicesss = map_cx(lambdass, indicesss, indices[0], indices[1])

print(lambdass[0])
print(indicesss[0])

[ 2.69106001e-02 -4.77333677e-02  1.52754293e-02  1.02444505e-01
  5.40356450e-02 -4.64070538e-02 -7.26300313e-02 -1.57385456e-01
 -9.09488381e-03  6.16242334e-03  1.08126000e-02  5.65799817e-02
 -1.33057828e-01  1.01779097e-01  1.80260456e-01  9.16121861e-03
 -1.46505912e-02 -8.44715406e-03 -3.40112356e-02  1.53502990e-02
  1.73467237e-02  4.00823784e-02 -9.21406669e-02  5.92633049e-03
 -2.50895970e-02  1.66444544e-02 -6.84040378e-03 -1.51895737e-02
  4.54278199e-02  3.40717608e-02 -3.35976492e-02  4.69102625e-03
  5.98828632e-02  2.74832274e-02  3.46188881e-03 -1.65124667e-02
  1.01649034e-02 -1.06291617e-01  1.08175235e-01  4.15851418e-03
 -4.95598970e-03 -5.47765498e-02  6.13869292e-02  4.56660188e-02
  1.80079981e-02 -3.68092866e-02  6.88370415e-02  3.84777388e-03
 -1.40948557e-02 -1.84339929e-02 -4.74180003e-02  7.25533556e-03
 -4.17919257e-02  2.62532329e-02  3.17322951e-02 -7.05871652e-03
 -5.91220021e-02  2.06097341e-02  6.91079461e-02 -9.56647665e-03
 -2.02186197e-02 -7.45951

In [2]:
lambdass[0][65]

array(-0.02203709)